In [208]:
import os
import yaml
import pyodbc
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

_join = os.path.join
toolpath = os.getcwd()

CONFIG = r'T:/ABM/ABM_FY21/Sketch_Planning/VisionEval/Model/Update_Automations/settings.yml'
with open(CONFIG) as cff:
    config =yaml.safe_load(cff)
scenario_list = config['scenario_list']
target_path = ''.join(config['target_path'])
ComSvcPow = config['ComSvcPow']
hvytrk_powertrain_prop = config['hvytrk_powertrain_prop']
ComSvcLtTrkProp = config['ComSvcLtTrkProp']

In [200]:
conn = pyodbc.connect("DRIVER={SQL Server};"
                      "SERVER=DDAMWSQL16;"
                      "DATABASE=dpoe_stage;" 
                      "Trusted_Connection=yes;")
sql2 = ("SELECT * FROM [dpoe_stage].[veh_reg_dmv].[fact] where yr=2017 and own = 'Commercial' and not make is null and not series is null")
df_sql2 = pd.read_sql_query(sql2, conn)
df_sql2['make'] = df_sql2['make'].str.lower()
df_sql2['series'] = df_sql2['series'].str.lower()
df_sql2['category'] = ""

In [201]:
df_comm = pd.read_csv(_join(toolpath, 'dmv commercial vehicle class dataset-lookup.csv'))
df_comm['make'] = df_comm['make'].str.lower()
df_comm['series'] = df_comm['series'].str.lower()
df_vehset = pd.read_csv(_join(toolpath, 'vehicle class dataset-lookup.csv'))
df_vehset['Make'] = df_vehset['Make'].str.lower()
df_vehset['Model'] = df_vehset['Model'].str.lower()

In [202]:
counter = 20000
df_dmvexcept = pd.DataFrame(columns=df_sql2.columns)
for ind in df_sql2.index:
    if ind < 100000000000000000000001:
        df_select = df_vehset[df_vehset['Make'].str.contains(df_sql2['make'][ind])]
        df_commsel = df_comm[df_comm['make'].str.contains(df_sql2['make'][ind])]        
        if not df_select.empty:
            #process.extractOne(df_sql2['series'][ind], df_select['Model'])           
            try:
                item = process.extractOne(df_sql2['series'][ind], df_select['Model'])
                df_sql2['category'][ind] = df_select.loc[[item[2]]]['Category3'].to_list()[0]
                #print ('a111', df_select.loc[[item[2]]]['Category3'].to_list()[0])
            except:
                print ('a222', df_sql2['model'][ind])
        elif not df_commsel.empty:
                #process.extractOne(df_sql2['series'][ind], df_commsel['series'])           
            try:
                item = process.extractOne(df_sql2['series'][ind],  df_commsel['series'])
                df_sql2['category'][ind] = df_commsel.loc[[item[2]]]['category'].to_list()[0]
                #print ('a333',  df_commsel.loc[[item[2]]]['category'].to_list()[0])
            except:
                print ('a444', df_sql2['model'][ind]) 
        else:
            df_dmvexcept = df_dmvexcept.append(df_sql2.loc[[ind]])
            print ('a555')
        if ind % counter == 0:
            print ("processed", ind)

<ipython-input-202-9cfb5aad7185>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sql2['category'][ind] = df_select.loc[[item[2]]]['Category3'].to_list()[0]


processed 0


<ipython-input-202-9cfb5aad7185>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sql2['category'][ind] = df_commsel.loc[[item[2]]]['category'].to_list()[0]


processed 20000
processed 40000
processed 60000
processed 80000
processed 100000
processed 120000
processed 140000
processed 160000
processed 180000
processed 200000
processed 220000
processed 240000


In [203]:
def my_fuel(x):
    names = {
        'Ice': x[(x['fuel_type']=='COMPRESSED NGASOLINE')|(x['fuel_type']=='DIESEL')|(x['fuel_type']=='FLEX METHANOL')|(x['fuel_type']=='GASOLINE')|(x['fuel_type']=='NATURAL GAS')|(x['fuel_type']=='PROPANE')|(x['fuel_type']=='HYDROGENATURAL GAS')]['count'].sum(),
        'Hev': x[(x['fuel_type']=='FXELECTRIC')|(x['fuel_type']=='HGASOLINE')|(x['fuel_type']=='PHEV')]['count'].sum(),
        'Bev': x[x['fuel_type']=='ELECTRIC']['count'].sum()}
    return pd.Series(names)

# region_comsvc_powertrain_prop.csv
df_comsvc_powertrain = df_sql2.groupby(['category', 'fuel_type'])['dmv_registration_id'].count().reset_index(name ='count')
df_car = df_comsvc_powertrain.loc[df_comsvc_powertrain['category'] == 'car']
df_lt = df_comsvc_powertrain.loc[df_comsvc_powertrain['category'] == 'light truck']
df_car = df_car.groupby('count').apply(my_fuel)         
df_lt = df_lt.groupby('count').apply(my_fuel)   
carList = df_car.sum(axis = 0, skipna = True).to_list()
carperList = [round(i/sum(carList),4)  for i in carList]
ltList = df_lt.sum(axis = 0, skipna = True).to_list()
ltperList = [round(i/sum(ltList),4)  for i in ltList]
list_comsvc_powertrain = carperList + ltperList
df_compow = pd.read_csv(_join(target_path, 'inputs', 'region_comsvc_powertrain_prop.csv'))
df_new = pd.DataFrame(columns = df_compow.columns.to_list())
i = 0
for row in ComSvcPow:
    if i == 0:
         row = [scenario_list[i][0]] + list_comsvc_powertrain
    else:
         row = [scenario_list[i][0]] + row
    df_new.loc[i] = row 
    i += 1
df_new['Year'] = df_new['Year'].astype('int64')
df_new.to_csv(_join(target_path, 'inputs', 'region_comsvc_powertrain_prop.csv'), index = False)  

In [204]:
# region_hvytrk_powertrain_prop.csv
df_ht = df_comsvc_powertrain.loc[df_comsvc_powertrain['category'] == 'Heavy Truck']
df_ht = df_ht.groupby('count').apply(my_fuel)   
htList = df_ht.sum(axis = 0, skipna = True).to_list()
htperList = [round(i/sum(htList),4)  for i in htList]
df_hvypow = pd.read_csv(_join(target_path, 'inputs', 'region_hvytrk_powertrain_prop.csv'))
df_new = pd.DataFrame(columns = df_hvypow.columns.to_list())
i = 0
for row in hvytrk_powertrain_prop:
    if i == 0:
         row = [scenario_list[i][0]] + htperList
    else:
         row = [scenario_list[i][0]] + row
    df_new.loc[i] = row 
    i += 1
df_new['Year'] = df_new['Year'].astype('int64')
df_new.to_csv(_join(target_path, 'inputs', 'region_hvytrk_powertrain_prop.csv'), index = False)  


In [219]:
#region_comsvc_lttrk_prop.csv
df_vehtype = df_sql2.groupby(['category'])['dmv_registration_id'].count().reset_index(name ='count')
df_vehtype['percent'] = (round(df_vehtype['count'] / df_vehtype['count'].sum(),2))
df_comlt = pd.read_csv(_join(target_path, 'inputs', 'region_comsvc_lttrk_prop.csv'))
df_new = pd.DataFrame(columns = df_comlt.columns.to_list())
i = 0
lt_base_persent = df_vehtype['percent'].loc[df_vehtype['category'] == 'light truck'].to_list()
for row in ComSvcLtTrkProp:
    if i == 0:
         row = [scenario_list[i][0]] + lt_base_persent
    else:
         row = [scenario_list[i][0]] + [row]
    df_new.loc[i] = row 
    i += 1
df_new.to_csv(_join(target_path, 'inputs', 'region_comsvc_lttrk_prop.csv'), index = False)   